In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture


In [2]:
df = pd.read_excel("../data/champions_group_data.xlsx")

# Quick overview
print(df.shape)
df.head()


(8559, 72)


,DUNS Number,Company Sites,Website,Address Line 1,City,State,State Or Province Abbreviation,Postal Code,Country,Phone Number,...,ISIC Rev 4 Code,ISIC Rev 4 Description,IT Budget,IT spend,No. of PC,No. of Desktops,No. of Laptops,No. of Routers,No. of Servers,No. of Storage Devices
0,639677726,"Zyf Lopsking Material Technology Co., Ltd. No....",NaN,"No.2777 Taidong Road, Panyang Industrial Park,...",Suzhou,Jiangsu,JS,215000.0,CHINA,NaN,...,4662.0,Wholesale of metals and metal ores,0,0,1 to 10,1 to 10,1 to 10,1 to 10,NaN,NaN
1,547756179,"Beijing Kaishi Lide Commerce And Trade Co., Lt...",NaN,"No.306, Building 8, Shuangyushudongli Jia, Hai...",Beijing,Beijing,BJ,100086.0,CHINA,1.082129e+09,...,NaN,NaN,5601,3472,1 to 10,1 to 10,1 to 10,1 to 10,1 to 10,1 to 10
2,728834216,Keshan Shengren Potato Industry Processing Co....,NaN,"South of Siduan Road, Xi Street, Keshan Town, ...",Qiqihar,Heilongjiang,HL,161000.0,CHINA,NaN,...,NaN,NaN,86905,53881,11 to 50,1 to 10,1 to 10,1 to 10,1 to 10,1 to 10
3,728791839,Zuoquan County Yuanfeng Agriculture Technology...,NaN,"Songaoliang Village, Liaoyang Town, Zuoquan Co...",Jinzhong,Shanxi,SX,30600.0,CHINA,NaN,...,NaN,NaN,11630,7210,NaN,NaN,NaN,NaN,NaN,NaN
4,728889244,Zuoquan County Tianxin Real Estate Development...,NaN,"No.14, Chengyadao Alley, Zuoquan County ...",Jinzhong,Shanxi,SX,30600.0,CHINA,3.548653e+09,...,NaN,NaN,94564,58629,1 to 10,1 to 10,1 to 10,1 to 10,1 to 10,1 to 10


# Data cleaning

In [3]:
df.columns = (
    df.columns
    .str.strip()              # remove leading/trailing spaces
    .str.replace('\n', ' ')   # remove line breaks
    .str.replace('\r', ' ')
)

In [4]:
def range_to_midpoint(x):
    """
    Convert range strings like '51-100' or '100+' to numeric midpoints
    """
    if pd.isna(x):
        return np.nan
    
    x = str(x).replace(",", "").strip()
    
    if "-" in x:
        low, high = x.split("-")
        return (float(low) + float(high)) / 2
    elif "+" in x:
        return float(x.replace("+", "")) * 1.2  # conservative assumption
    else:
        try:
            return float(x)
        except:
            return np.nan



In [5]:
numeric_cols = [
    "Employees Total",
    "Revenue (USD)",
    "IT Budget",
    "IT spend",
    "No. of PC",
    "No. of Desktops",
    "No. of Laptops",
    "No. of Routers",
    "No. of Servers",
    "No. of Storage Devices"
]

for col in numeric_cols:
    df[col] = df[col].apply(range_to_midpoint)


In [6]:
hardware_cols = [
    "No. of PC",
    "No. of Desktops",
    "No. of Laptops",
    "No. of Routers",
    "No. of Servers",
    "No. of Storage Devices"
]

df["hardware_footprint"] = df[hardware_cols].sum(axis=1)


# Feature engineering

In [7]:
# Avoid divide-by-zero
df["employees"] = df["Employees Total"].replace(0, np.nan)

# ① IT Intensity
df["it_spend_per_employee"] = df["IT spend"] / df["employees"]

# ② Hardware Density
df["hardware_per_employee"] = df["hardware_footprint"] / df["employees"]

# ③ Spend–Hardware Alignment
df["it_spend_per_hardware"] = df["IT spend"] / df["hardware_footprint"]


In [8]:
for col in [
    "it_spend_per_employee",
    "hardware_per_employee",
    "it_spend_per_hardware"
]:
    df[col] = df[col].clip(df[col].quantile(0.01), df[col].quantile(0.99))


c:\Users\12612\Downloads\DSA2101\src\venv-dsa2101\Lib\site-packages\numpy\lib\_function_base_impl.py:4671: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
c:\Users\12612\Downloads\DSA2101\src\venv-dsa2101\Lib\site-packages\numpy\lib\_function_base_impl.py:4671: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


# EDA

In [9]:
features = [
    "it_spend_per_employee",
    "hardware_per_employee",
    "it_spend_per_hardware"
]

df[features].describe()


c:\Users\12612\Downloads\DSA2101\src\venv-dsa2101\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\12612\Downloads\DSA2101\src\venv-dsa2101\Lib\site-packages\numpy\lib\_function_base_impl.py:4671: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,it_spend_per_employee,hardware_per_employee,it_spend_per_hardware
count,5427.000000,5427.0,5240.0
mean,5001.200104,0.0,inf
std,12551.719817,0.0,NaN
min,0.000000,0.0,inf
25%,210.687500,0.0,NaN
50%,1841.850000,0.0,NaN
75%,4239.560000,0.0,NaN
max,95448.200000,0.0,inf


# Clustering

In [11]:
from sklearn.impute import SimpleImputer


In [13]:
# IMPORTANT: rebuild X from original df
X = df[features].copy()

print("Before cleaning:", X.shape)


Before cleaning: (8559, 3)


In [14]:
for col in X.columns:
    X[col] = pd.to_numeric(X[col], errors="coerce")


In [15]:
X.replace([np.inf, -np.inf], np.nan, inplace=True)


In [16]:
print(X.shape)
print(X.isna().mean())


(8559, 3)
it_spend_per_employee    0.365931
hardware_per_employee    0.365931
it_spend_per_hardware    1.000000
dtype: float64


In [17]:
X = X.dropna(axis=1, how="all")


In [18]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler


In [19]:
imputer = SimpleImputer(strategy="median")
X_imputed = imputer.fit_transform(X)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

In [20]:
kmeans = KMeans(n_clusters=4, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

X["cluster"] = clusters

In [21]:
cluster_summary = X.groupby("cluster").mean()
cluster_summary

,it_spend_per_employee,hardware_per_employee
cluster,,
0,2470.424443,0.0
1,92360.956384,0.0
2,53386.831190,0.0
3,21440.877454,0.0


In [22]:
def label_it_structure(row):
    if row["hardware_per_employee"] > X["hardware_per_employee"].median() and \
       row["it_spend_per_hardware"] < X["it_spend_per_hardware"].median():
        return "Hardware-centric"
    
    elif row["it_spend_per_employee"] > X["it_spend_per_employee"].median() and \
         row["hardware_per_employee"] < X["hardware_per_employee"].median():
        return "Software / Services-oriented"
    
    elif row["it_spend_per_employee"] < X["it_spend_per_employee"].quantile(0.25) and \
         row["hardware_per_employee"] < X["hardware_per_employee"].quantile(0.25):
        return "Under-invested"
    
    else:
        return "Balanced"


In [23]:
cluster_labels = {}

for c in cluster_summary.index:
    cluster_labels[c] = label_it_structure(cluster_summary.loc[c])

X["IT_Investment_Structure"] = X["cluster"].map(cluster_labels)


In [ ]:
df = df.merge(
    X[["cluster", "IT_Investment_Structure"]],
    left_index=True,
    right_index=True,
    how="left"
)

In [25]:
df["IT_Investment_Structure"].value_counts()

IT_Investment_Structure
Balanced    8559
Name: count, dtype: int64